# Introduction
---

# Import libries

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the data

# Build the model

In [ ]:
class CNN(nn.Module):
    def __init__(self, n_classes):
        super(CNN, self).__init__()
        self.pipe1 = nn.Sequential(nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=2), 
                                   nn.ReLU(), 
                                   nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
                                  )
        self.pipe2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=2), 
                                   nn.ReLU(), 
                                   nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
                                  )
        self.pipe3 = nn.Sequential(nn.Linear(), nn.ReLU(), nn.Linear(,1), nn.Sigmoid())